Purpose:
* Getting basic info about pubmed XML structure.
* Testing scripts for parsing XML files from pubmed.

In [1]:
# Set working directory
from pathlib import Path 
wd = Path('/mnt/d/project/plant_sci_history/1_obtaining_corpus/_test')

## Traget tags

For each article:
* `ISOAbbreviation` for journal name
* `ArticleTitle`
* `AbstractText`
* `PubMedPubDate PubStatus="pubmed"`: to flag the start of date info
  * `Year`
  * `Month`
  * `Day`
* `ArticleId IdType="pubmed"`: this holds PMID

In [10]:
AR = "PubmedArticle"    # new article
TI = "ArticleTitle"     # title begin tag
AB = "AbstractText"
JO = "ISOAbbreviation"
DA = "PubMedPubDate PubStatus=\"pubmed\""
DAe= "/PubMedPubDate"   # Note that other PubStatus also has the
                        # same end tag.
YR = "Year"
MO = "Month"
DY = "Day"
PM = "ArticleId IdType=\"pubmed\"" 

In [12]:
# Rid of the tags
def get_value(L):
    L = L.split(">")[1]
    L = L.split("</")[0]
    return L

In [32]:
# {pmid:{"TI":title,
#        "DA":["yyyy","mm","dd"],
#        "AB":abstract",
#        "JO":journal_name}
pubmed_d = {}

# read file line by line
test_xml  = "test.xml"
input_obj = open(wd / test_xml, "r")
L         = input_obj.readline()
c         = 0
fields    = ["","","","","",""] # [TI, AB, JO, YR, MO, DY] 

# whether DA_b flag is found or not before encoutering an DA end tag.
flag_DA   = 0
PMID      = ""
while L != "":
    L = L.strip()
    if L.startswith(f"<{AR}>") != 0:
        # This is set to make sure PMID tag is found for this article
        # and values are stored in the dictionary.
        fields   = ["","","","","",""]
        PMID = ""
        #print("\n",L)
        c+= 1
    elif L.startswith(f"<{TI}") != 0:
        fields[0] = get_value(L)
    elif L.startswith(f"<{AB}") != 0:
        fields[1] = get_value(L) 
    elif L.startswith(f"<{JO}") != 0:
        fields[2] = get_value(L)
    elif L.startswith(f"<{DA}") != 0:
        flag_DA = 1
    elif L.startswith(f"<{YR}") != 0 and flag_DA == 1:
        fields[3] = get_value(L)
    elif L.startswith(f"<{MO}") != 0 and flag_DA == 1:
        fields[4] = get_value(L)
    elif L.startswith(f"<{DY}") != 0 and flag_DA == 1:
        fields[5] = get_value(L)
    # Encouter Date end tag when a corresponding begin tag exists
    elif L.startswith(f"<{DAe}") != 0 and flag_DA == 1:
        flag_DA = 0
    elif L.startswith(f"<{PM}") != 0:
        PMID = get_value(L)
        if PMID not in pubmed_d:
            pubmed_d[PMID] = fields
        else:
            print("ERR: redundant PMID,", PMID)
        #print("PMID:",PMID)
        #print(fields)
    L = input_obj.readline()
           
print("# articles:",c)

# articles: 5


## Test filtering to get plant science articles

Assumptions about a plant science article:
* Journal name contains any of the following:
  * plant, plants, botany, botanical
* Or title/abstract mentioned any plant taxonomic terms all the way to the genus level. 

nodes.dmp file consists of taxonomy nodes. The description for each node includes the following columns:
* tax_id -- node id in GenBank taxonomy database
* parent tax_id -- parent node id in GenBank taxonomy database
* rank -- rank of this node (superkingdom, kingdom, ...)
* embl code -- locus-name prefix; not unique
* division id -- see division.dmp file
* inherited div flag (1 or 0) -- 1 if node inherits division from parent
* genetic code id -- see gencode.dmp file
* inherited GC flag (1 or 0) -- 1 if node inherits genetic code from parent
* mitochondrial genetic code id -- see gencode.dmp file
* inherited MGC flag (1 or 0) -- 1 if node inherits mitochondrial gencode from parent
* GenBank hidden flag (1 or 0) -- 1 if name is suppressed in GenBank entry lineage
* hidden subtree root flag (1 or 0) -- 1 if this subtree has no sequence data yet
* comments-- free-text comments and citations

In [36]:
target    = "Viridiplantae"
wd        = Path("/mnt/d/project/plant_sci_history/1_obtaining_corpus/taxonomy")
names_dmp = open(wd / "names.dmp")
L         = names_dmp.readline()
while L != "":
    print(L.strip().split("\t"))
    L = names_dmp.readline()



TypeError: unsupported operand type(s) for /: 'str' and 'str'

## Continue development in script_parse_pubmed.py